# 01_source_raw — OpenBDAP Saldi Storico Stato

## 📌 Obiettivo

Questo notebook scarica e archivia il dataset **RAW** relativo ai:

> Rendiconto pubblicato – Serie storica – Saldi  
> Fonte: OpenBDAP (RGS – Ragioneria Generale dello Stato)

L'obiettivo è:

- Scaricare il dataset originale (senza trasformazioni)
- Salvare uno snapshot versionato su Google Drive
- Calcolare SHA256 per garantire integrità
- Generare un file `run_manifest.json` per la tracciabilità del run

---

## 🔎 Dataset

- Fonte: https://bdap-opendata.rgs.mef.gov.it
- Dataset: Rendiconto pubblicato – Serie storica – Saldi
- Periodo: 2003–2024
- Livello: Anno

---

## 📂 Output

Il notebook genera la seguente struttura su Google Drive:

```
MyDrive/
└── DataCivicLab/
    └── data/
        └── raw/
            └── openbdap_rendiconto_saldi_storico/
                └── <RUN_ID>/
                    ├── rendiconto_pubblicato_serie_storica_saldi_raw.csv
                    ├── run_manifest.json
                    └── profile_basic.json
```

Ogni esecuzione crea una nuova cartella `<RUN_ID>` in UTC.

---

## 🔐 Integrità dei dati

Per ogni run viene calcolato:

- SHA256 del file RAW
- Dimensione in bytes
- Timestamp UTC del download

Questo garantisce tracciabilità e riproducibilità.

---

## ⚠️ Nota importante

Il file salvato in questa fase è **RAW**:

- Nessuna pulizia
- Nessuna rinomina colonne
- Nessuna normalizzazione
- Nessuna modifica al contenuto originale

Qualsiasi trasformazione verrà effettuata nel notebook successivo `02_clean_transform.ipynb`.

In [15]:
from pathlib import Path
from datetime import datetime, timezone
import os, shutil, json, hashlib
import requests
import pandas as pd

# -------------------------
# 0) Mount Drive (robusto) + standard path /content/drive
# -------------------------
from google.colab import drive

MOUNT_REAL = "/content/gdrive"
os.makedirs(MOUNT_REAL, exist_ok=True)
drive.mount(MOUNT_REAL)

p_drive = Path("/content/drive")
if p_drive.exists() and not p_drive.is_symlink():
    shutil.rmtree(p_drive, ignore_errors=True)
if not p_drive.exists():
    os.symlink(MOUNT_REAL, "/content/drive")

assert Path("/content/drive/MyDrive").exists(), "MyDrive non trovato: mount non ok."

# -------------------------
# 1) CONFIG
# -------------------------
PROJECT = "openbdap_rendiconto_saldi_storico"
DATASET_SLUG = "rendiconto_pubblicato_serie_storica_saldi"

DATASET_PAGE = "https://bdap-opendata.rgs.mef.gov.it/content/rendiconto-pubblicato-serie-storica-saldi"
EXPORT_CSV_URL = "https://bdap-opendata.rgs.mef.gov.it/export/csv/Rendiconto-Pubblicato---Serie-storica---Saldi.csv"

ROOT = Path("/content/drive/MyDrive/DataCivicLab")
RUN_ID = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M%S")
RUN_DIR = ROOT / "data" / "raw" / PROJECT / RUN_ID
RUN_DIR.mkdir(parents=True, exist_ok=True)

RAW_CSV_PATH = RUN_DIR / f"{DATASET_SLUG}_raw.csv"
TMP_PATH = RUN_DIR / f"{DATASET_SLUG}_raw.__tmp__"
MANIFEST_PATH = RUN_DIR / "run_manifest.json"
PROFILE_PATH = RUN_DIR / "profile_basic.json"

HEADERS = {
    "User-Agent": "Mozilla/5.0",
    "Accept": "text/csv,application/csv,text/plain,*/*",
}

def looks_like_html(b: bytes) -> bool:
    s = b.lstrip()[:300].lower()
    return (s.startswith(b"<!doctype html") or s.startswith(b"<html") or b"<div" in s[:80] or b"<head" in s[:120])

# -------------------------
# 2) Download SAFE (temp -> validate -> final)
# -------------------------
r = requests.get(EXPORT_CSV_URL, headers=HEADERS, stream=True, timeout=180, allow_redirects=True)
print("HTTP:", r.status_code)
print("Final URL:", r.url)
print("Content-Type:", r.headers.get("Content-Type"))
r.raise_for_status()

h = hashlib.sha256()
bytes_written = 0

# grab first chunk to validate
it = r.iter_content(chunk_size=1024 * 1024)
first_chunk = next(it, b"") or b""

if looks_like_html(first_chunk):
    preview = first_chunk[:300].decode("utf-8", errors="replace")
    raise RuntimeError("Il download sembra HTML, non CSV. Preview:\n" + preview)

with open(TMP_PATH, "wb") as f:
    # write first chunk
    f.write(first_chunk)
    h.update(first_chunk)
    bytes_written += len(first_chunk)

    # write remaining
    for chunk in it:
        if not chunk:
            continue
        f.write(chunk)
        h.update(chunk)
        bytes_written += len(chunk)

sha256 = h.hexdigest()

# rename temp -> final atomically
if RAW_CSV_PATH.exists():
    RAW_CSV_PATH.unlink()
TMP_PATH.rename(RAW_CSV_PATH)

print("Saved RAW:", RAW_CSV_PATH)
print("Bytes:", bytes_written)
print("SHA256:", sha256)

# -------------------------
# 3) Quick sanity check (non modifica i dati)
# -------------------------
# stampa prime 5 righe per confermare che sia tabellare
lines = RAW_CSV_PATH.read_text(encoding="utf-8", errors="replace").splitlines()
print("\n--- First 5 lines ---")
for i, line in enumerate(lines[:5], start=1):
    print(f"{i:02d}: {line[:200]}")
print("Total lines:", len(lines))

# -------------------------
# 4) run_manifest.json
# -------------------------
manifest = {
    "project": PROJECT,
    "dataset_slug": DATASET_SLUG,
    "source": "OpenBDAP (RGS)",
    "dataset_page": DATASET_PAGE,
    "csv_url": EXPORT_CSV_URL,
    "final_url": r.url,
    "run_id": RUN_ID,
    "downloaded_at_utc": datetime.now(timezone.utc).isoformat(),
    "raw_file": {
        "path": str(RAW_CSV_PATH),
        "bytes": int(bytes_written),
        "sha256": sha256,
        "content_type": r.headers.get("Content-Type"),
    },
    "notes": "RAW CSV downloaded and stored as-is. No transformations applied."
}

with open(MANIFEST_PATH, "w", encoding="utf-8") as f:
    json.dump(manifest, f, ensure_ascii=False, indent=2)

print("\nSaved manifest:", MANIFEST_PATH)

# -------------------------
# 5) profile_basic.json (opzionale, non blocca se il CSV è strano)
# -------------------------
try:
    df = pd.read_csv(RAW_CSV_PATH, sep=None, engine="python", encoding="utf-8-sig", on_bad_lines="skip")
    profile_basic = {
        "rows": int(df.shape[0]),
        "cols": int(df.shape[1]),
        "columns": [str(c) for c in df.columns],
        "dtypes": {str(c): str(t) for c, t in df.dtypes.items()},
        "nulls": {str(c): int(df[c].isna().sum()) for c in df.columns},
        "profiling_notes": "Computed via pandas read_csv with sep autodetect and on_bad_lines='skip'. RAW file is untouched."
    }
    with open(PROFILE_PATH, "w", encoding="utf-8") as f:
        json.dump(profile_basic, f, ensure_ascii=False, indent=2)
    print("Saved profile:", PROFILE_PATH)
except Exception as e:
    print("Profile skipped due to parse issue:", type(e).__name__, str(e))

print("\nContents of RUN_DIR:")
!ls -lah "{RUN_DIR}"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
HTTP: 200
Final URL: https://bdap-opendata.rgs.mef.gov.it/export/csv/Rendiconto-Pubblicato---Serie-storica---Saldi.csv
Content-Type: text/csv
Saved RAW: /content/drive/MyDrive/DataCivicLab/data/raw/openbdap_rendiconto_saldi_storico/20260226_194250/rendiconto_pubblicato_serie_storica_saldi_raw.csv
Bytes: 8763
SHA256: 22bd23573f7193b27a5458932b5f1a04d84d5684a57691ac5e547e07a4ab7e5f

--- First 5 lines ---
01: "ANNO";"RISPARMIO_PUBBLICO";"SALDO_NETTO";"INDEBITAMENTO_NETTO";"RICORSO_MERCATO";"AVANZO_PRIMARIO";"SPESE_CORRENTI";"SPESE_INTERESSI";"SPESE_CONTO_CAPITALE";"SPESE_ACQ_ATT_FINE";"SPESE_RIMBORSO_PREST
02: "2003";"-23527106344.85";"-76608438560.85";"-70589525371.85";"-333428879750.18";"-469519795.74";"404491049044.85";"76138918765.11";"61602074235.00";"7116577049.00";"256820441189.33";"722913564469.18";
03: "2004";"-30341184650.00";"-63777783304.76";"-5792